In [1]:
from pathlib import Path
import pprint
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn import linear_model
import numpy as np
from scipy.stats import linregress
import statsmodels.api as sm
import itertools
import warnings
warnings.filterwarnings('ignore')

In [2]:
col_names=['x1','x2','x3','x4']
df = pd.read_csv('PCA.csv', 
                 sep=",", 
                 skiprows=0,
                 names=col_names)

- a) Plot the time series for each variable. [1 point for the plot(s)] 
- b) Perform PCA on the data. [1 point for correct size of input data and correct output of PCA] 
- c) Find which modes you want to keep in order to reconstruct the data and provide a rationale for your selection. [1 point for the choice of modes and 1 point for the rationale]. 
- d) Plot the PCs of the significant modes (i.e. those that you kept) in time. Briefly discuss the results (what are these plots telling you?) [1 point for the plot(s) and 1 point for the discussion.] 
- e) Plot PC1 vs PC2.  Discuss any feature that you find interesting.  [1 point for the plot, 1 point for the discussion.] 
 

### Plot the time series for each variable